# Exploratory Data Analysis

## Import

In [1]:
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import sys
import os
#from rapidfuzz import fuzz, process
import unidecode
#import requests
import json

## Raw Data

In [2]:
df_movie = pd.read_csv("../data/raw/TMDB_movie_dataset_v11.csv")
df_movie.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304590 entries, 0 to 1304589
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1304590 non-null  int64  
 1   title                 1304574 non-null  object 
 2   vote_average          1304590 non-null  float64
 3   vote_count            1304590 non-null  int64  
 4   status                1304590 non-null  object 
 5   release_date          1044077 non-null  object 
 6   revenue               1304590 non-null  int64  
 7   runtime               1304590 non-null  int64  
 8   adult                 1304590 non-null  bool   
 9   backdrop_path         333216 non-null   object 
 10  budget                1304590 non-null  int64  
 11  homepage              135618 non-null   object 
 12  imdb_id               642282 non-null   object 
 13  original_language     1304590 non-null  object 
 14  original_title        1304574 non-

## Checking the relevant data

- Removendo campos que não irão ser relevantes a nossas métricas.

In [ ]:
df_movie.drop(columns=['poster_path','tagline','imdb_id','homepage','backdrop_path','overview','spoken_languages'],inplace=True)
df_movie = df_movie.loc[~(df_movie['adult'] == True)]
df_movie = df_movie.loc[(df_movie['status'] == 'Released')]
df_movie.drop(columns=['adult'],inplace=True)
df_movie.drop(columns=['status'],inplace=True)
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1136081 entries, 0 to 1304589
Data columns (total 16 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1136081 non-null  int64  
 1   title                 1136070 non-null  object 
 2   vote_average          1136081 non-null  float64
 3   vote_count            1136081 non-null  int64  
 4   release_date          915020 non-null   object 
 5   revenue               1136081 non-null  int64  
 6   runtime               1136081 non-null  int64  
 7   budget                1136081 non-null  int64  
 8   original_language     1136081 non-null  object 
 9   original_title        1136070 non-null  object 
 10  popularity            1136081 non-null  float64
 11  genres                704350 non-null   object 
 12  production_companies  446399 non-null   object 
 13  production_countries  592814 non-null   object 
 14  spoken_languages      623131 non-null  

## Null Values and Filtering

- Removendo items com ids duplicados.
- Removendo entradas sem 
- Irei separar as keywords em outro dataset apenas para word cloud and word count.

In [ ]:
df_movie = df_movie.drop_duplicates(subset='id', keep='first')
df_movie = df_movie.loc[~df_movie['title'].isna()]
df_keywords = df_movie['keywords'].dropna()
df_movie.drop(columns=['keywords'],inplace=True)
df_movie['release_date'] = pd.to_datetime(df_movie['release_date'], errors='coerce')
df_movie.isnull().sum()

id                           0
title                        0
vote_average                 0
vote_count                   0
release_date            220492
revenue                      0
runtime                      0
budget                       0
original_language            0
original_title               0
popularity                   0
genres                  431122
production_companies    688981
production_countries    542574
dtype: int64

In [42]:
df_movie.loc[~
    ((df_movie['revenue'] == 0) | (df_movie['budget'] == 0))
    ]

,id,title,vote_average,vote_count,release_date,revenue,runtime,budget,original_language,original_title,popularity,genres,production_companies,production_countries
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,160000000,en,Inception,83.952,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America"
1,157336,Interstellar,8.417,32571,2014-11-05,701729206,169,165000000,en,Interstellar,140.241,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America"
2,155,The Dark Knight,8.512,30619,2008-07-16,1004558444,152,185000000,en,The Dark Knight,130.643,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America"
3,19995,Avatar,7.573,29815,2009-12-15,2923706026,162,237000000,en,Avatar,79.932,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom"
4,24428,The Avengers,7.710,29166,2012-04-25,1518815515,143,220000000,en,The Avengers,98.082,"Science Fiction, Action, Adventure",Marvel Studios,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300293,809904,Old Home on the Corner,0.000,0,2021-03-03,250,2,100,en,Old Home on the Corner,0.600,"Drama, Documentary, Fantasy",Love Streams Films,United States of America
1300885,817609,Tatlı Bela,0.000,0,1961-01-01,100000,0,75000,tr,Tatlı Bela,1.400,Adventure,Güven Film,Turkey
1300906,817595,The Last Draw,0.000,0,NaN,20000,90,100,en,The Last Draw,0.600,NaN,NaN,NaN
1301488,816771,Granny: House of Fire,0.000,0,NaN,15,18,15,en,Granny: House of Fire,0.600,"Horror, Action",NaN,NaN


> - Podemos notar, a principio, que temos 15 mil entradas com informações financeiras completas.
> - Irei verificar se existem entradas relevantes fora dessa amostragem.

## Subsets

- Criando tabelas de estatísticas antes de filtrar os filmes por popularidade.

In [58]:
filtered_rows = len(df_movie.loc[
    ((df_movie['popularity'] < 1.5) & (df_movie['vote_count'] < 25)) &
    ((df_movie['revenue'] == 0) | (df_movie['budget'] == 0))
    ].sort_values("popularity", ascending=False))

total_rows = len(df_movie)
percentage = (filtered_rows / total_rows) * 100

df_statistics = pd.DataFrame({
    'Métrica': ['Total', 'Engajamento Nulo'],
    'Quantidade': [total_rows, filtered_rows],
    'Percentual (%)': [100, round(percentage, 2)]
})

del percentage
del total_rows
del filtered_rows

- Dataframe contando generos e companias 

In [111]:
df_genres_count = df_movie['genres'].str.split(', ').explode().value_counts().reset_index()
df_genres_count


,genres,count
0,Drama,229583
1,Documentary,177028
2,Comedy,142705
3,Animation,61077
4,Horror,54866
5,Music,52450
6,Romance,51230
7,Thriller,46767
8,Action,44370
9,Crime,33630


In [131]:
df_companies_count = df_movie['production_companies'].str.split(', ').explode().value_counts().reset_index()
df_companies_count = df_companies_count.loc[df_companies_count['count'] > 50]
df_companies_count

,production_companies,count
0,BBC,3307
1,Warner Bros. Pictures,3162
2,Columbia Pictures,2910
3,Metro-Goldwyn-Mayer,2854
4,ONF | NFB,2807
...,...,...
1318,Butcher's Film Service,51
1319,Two Cities Films,51
1320,BeTV,51
1321,Yoshimoto Kogyo,51


In [106]:
df_duration = df_movie.loc[df_movie['runtime'] > 60,'runtime']
df_duration

0          148
1          169
2          152
3          162
4          143
          ... 
1304579    190
1304580    170
1304583     95
1304588     80
1304589     70
Name: runtime, Length: 376710, dtype: int64

In [94]:
df_movie.loc[
    ((df_movie['popularity'] > 1.0) | (df_movie['vote_count'] > 20)) &
    ((df_movie['revenue'] > 0) | (df_movie['budget'] > 0))
    ]

,id,title,vote_average,vote_count,release_date,revenue,runtime,budget,original_language,original_title,popularity,genres,production_companies,production_countries
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,160000000,en,Inception,83.952,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America"
1,157336,Interstellar,8.417,32571,2014-11-05,701729206,169,165000000,en,Interstellar,140.241,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America"
2,155,The Dark Knight,8.512,30619,2008-07-16,1004558444,152,185000000,en,The Dark Knight,130.643,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America"
3,19995,Avatar,7.573,29815,2009-12-15,2923706026,162,237000000,en,Avatar,79.932,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom"
4,24428,The Avengers,7.710,29166,2012-04-25,1518815515,143,220000000,en,The Avengers,98.082,"Science Fiction, Action, Adventure",Marvel Studios,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302296,818285,My Brother's Keeper,0.000,0,2021-04-01,0,97,7000,en,My Brother's Keeper,1.284,Thriller,Victory Productions,United States of America
1302777,814029,White Terror,0.000,0,2020-07-31,0,80,750000,en,White Terror,1.480,"Drama, Horror, Mystery, Thriller","Error 404 Productions, Seven Look Productions",United States of America
1303954,816654,Greta,0.000,0,2021-04-09,0,8,600,en,Greta,1.130,Drama,NaN,United States of America
1304070,816450,Black Friday Subliminal,0.000,0,2021-05-28,0,86,2500000,en,Black Friday Subliminal,1.018,"Horror, Mystery",Robin Troja Films,United States of America


- Dlist por década

In [ ]:
def get_season(date):
    if pd.isnull(date):
        return None
    month = date.month
    if month in [12, 1, 2]:
        return 'Verão'
    elif month in [3, 4, 5]:
        return 'Outono'
    elif month in [6, 7, 8]:
        return 'Inverno'
    else:
        return 'Primavera'

df_movie['decade'] = ((df_movie['release_date'].dt.year // 10) * 10).astype('Int64')
df_movie['season'] = df_movie['release_date'].apply(get_season)
df_movie.loc[df_movie['decade'] < 1900, 'decade'] = 1000 #Antes de 1900
df_movie


,id,title,vote_average,vote_count,release_date,revenue,runtime,budget,original_language,original_title,popularity,genres,production_companies,production_countries,decade,season
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,160000000,en,Inception,83.9520,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America",2010,Inverno
1,157336,Interstellar,8.417,32571,2014-11-05,701729206,169,165000000,en,Interstellar,140.2410,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",2010,Primavera
2,155,The Dark Knight,8.512,30619,2008-07-16,1004558444,152,185000000,en,The Dark Knight,130.6430,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America",2000,Inverno
3,19995,Avatar,7.573,29815,2009-12-15,2923706026,162,237000000,en,Avatar,79.9320,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom",2000,Verão
4,24428,The Avengers,7.710,29166,2012-04-25,1518815515,143,220000000,en,The Avengers,98.0820,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,2010,Outono
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304583,814885,Away Bus,0.000,0,2019-04-20,0,95,0,en,Away Bus,0.6000,Drama,NaN,Ghana,2010,Outono
1304584,814886,L is for Lesbian,0.000,0,2020-01-01,0,4,0,pt,O L é de Lésbica,0.6000,NaN,NaN,NaN,2020,Verão
1304585,814887,Paradise,0.000,0,NaT,0,6,0,en,Paradise,0.6000,NaN,NaN,NaN,<NA>,None
1304588,814891,Howl,0.000,0,2021-04-05,0,80,0,en,Howl,0.9400,Horror,NaN,NaN,2020,Outono


- Criando dataframe relacionando a decada e season.

In [ ]:
df_decade_by_season = df_movie.groupby(
    ['decade', 'season']
    ).size().reset_index(name='count').pivot(
        index='decade', columns='season', values='count').fillna(0).astype(int).reindex(columns=[
            'Verão', 'Outono', 'Inverno', 'Primavera'
            ]).reset_index()

df_decade_by_season = df_decade_by_season.loc[~
((df_decade_by_season['decade'] == 2030) | (df_decade_by_season['decade'] == 2050) | (df_decade_by_season['decade'] == 2060))
]

df_decade_by_season['decade'] = df_decade_by_season['decade'].astype(str)
df_decade_by_season.loc[df_decade_by_season['decade'] == '1000','decade'] = 'Antes de 1900'
df_decade_by_season['decade'] = df_decade_by_season['decade'] + 's'

df_decade_by_season

season,decade,Verão,Outono,Inverno,Primavera
0,Antes de 1900s,768,502,344,489
1,1900s,1529,764,553,772
2,1910s,4920,2893,2718,3230
3,1920s,5332,3121,2532,3733
4,1930s,6828,4351,3554,4575
5,1940s,6738,3587,3332,3953
6,1950s,10761,4847,4580,5189
7,1960s,18600,6909,6236,6897
8,1970s,25457,9242,7730,8948
9,1980s,29452,10265,9360,10561


## Normalization

## Export

In [ ]:
df_genres_count.to_csv("../data/processed/genres_count.csv", index=False)
df_companies_count.to_csv("../data/processed/companies_count.csv", index=False)
df_duration.to_csv("../data/processed/duration_list.csv", index=False)
df_decade_by_season.to_csv("../data/processed/decade_by_season.csv", index=False)
